**Downloading Dataset**

In [1]:
# !pip install kaggle
# !mkdir /root/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# from google.colab import drive

In [ ]:
# drive.mount("./drive")

In [4]:
# !cd "./drive/MyDrive/FDD Project/dataset"

In [5]:

# !kaggle datasets download -d smaranjitghose/corn-or-maize-leaf-disease-dataset

In [6]:
# !unzip "/content/drive/MyDrive/FDD Project/dataset/corn-or-maize-leaf-disease-dataset.zip"

**Importing LIbraries**

In [ ]:
# !pip install split-folders
# ! pip install python_splitter

In [4]:
import tensorflow as tf
import numpy as np
import splitfolders
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Preprocessing Data**

In [5]:
# splitfolders.ratio("./data",
#                    output="./corn",
#                    seed=42,
#                    ratio=(.8, .0, .2)
#                    )

In [3]:
train_dir = "./corn/train"
test_dir = "/corn/test"

In [4]:
gen = ImageDataGenerator(rescale=1./255,
                         validation_split=0.2,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.3,
                         rotation_range=20,
                         fill_mode="nearest")

In [5]:
train_generator= gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    class_mode = "categorical",
    subset="training"
)


Found 2681 images belonging to 4 classes.


In [6]:
val_generator = gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    class_mode= "categorical",
    subset="validation"
)

Found 667 images belonging to 4 classes.


**Model Building**

In [7]:
base_model = VGG16(weights="imagenet", include_top=None)
base_model.trainable = False

In [3]:
inputs = tf.keras.layers.Input(shape = (224,224,3), name= "input_layer")
x = base_model(inputs)
print(x.shape)
x = layers.Dense(1024, activation="relu", name="dense_1")(x)
x = layers.MaxPool2D((2,2), name= "max_pool_layer")(x)
x= layers.Dense(256, activation="relu", name="dense_2")(x)
prediction = layers.Dense(4, activation= "softmax", name="output_layer")(x)

NameError: name 'tf' is not defined

In [9]:
model = models.Model(inputs, outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 dense_1 (Dense)             (None, 7, 7, 1024)        525312    
                                                                 
 max_pool_layer (MaxPooling  (None, 3, 3, 1024)        0         
 2D)                                                             
                                                                 
 dense_2 (Dense)             (None, 3, 3, 256)         262400    
                                                                 
 output_layer (Dense)        (None, 3, 3, 4)           1028      
                                                             

In [11]:
model.compile(optimizer= tf.keras.optimizers.Adam(),
              loss= losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"]
              )

In [2]:
history = model.fit(train_generator,
                    batch_size=len(train_generator),
                    epochs=50,
                    validation_data=val_generator,
                    validation_steps=int(0.25 * len(val_generator))
                    )

NameError: name 'model' is not defined

In [ ]:
img_path= "/content/lap.jpg"
img = image.load_img(img_path, target_size= (224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
features= model.predict(x)